**US Presidental Elections Analysis**

As in the heat of the US election, I make this this project to crush data of the president elections in the US, to draw some insights of the US top politic over presidental elctions over the past 4 decades. By the end, i will also visualise the US economy data with politician background. The project will be focus in parts:

State-level popular vote trend over elections
Nation-lavel popular vs electrol college vote margins over elections
Voter turn-out rate over time
The background of Polictic to the economy

In [ ]:
#Loading libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
#Loading datasets
#Election results at state-level 1976-2016 (Source:https://electionlab.mit.edu/data)
elections = pd.read_csv('../input/us-elections-dataset/president-1976-2016.csv')

#2020 Presidental election at county-level result-incomplted(Source: https://www.kaggle.com/unanimad/us-election-2020 Version 61st)
this_year_data = pd.read_csv('../input/us-1976-2020-statelevel-presidential-elections/USA/2020president_county_candidate.csv')

#Voter turnout-rate(Source - Wikipedia)
turnout = pd.read_csv('../input/us-1976-2020-statelevel-presidential-elections/USA/turnout_rate.csv')
#US domographic indexes (Source: WorldBank Data)
dmg = pd.read_csv('../input/us-1976-2020-statelevel-presidential-elections/USA/USA_demographic.csv')

# electoral college votes by states 
electrol = pd.read_csv('../input/us-1976-2020-statelevel-presidential-elections/USA/electoral_vote.csv')

1. Methodology

The presidential election in the US is popularly known about the competitive of 2 candidates. However, the fact is, there are more than 2 candidates in this final round. They come from smaller parties than the famous Republican and Democratic parties. To make the analysis more concentrate, I will only focus on these 2 parties which get the most support from American voters. Republic and Democrat. The race to the white house is not only about getting more public interest than the challenger but also reach the threshold of 270 Electrol college votes. Each state has a fixed number of college votes, and in all states but Maine and Nebraska, if a candidate wins popular, he gets all the electoral college. In this analysis, I will not dig into the details of election policies change over the years, but focus on popular votes and assume electoral votes by:

Maine and Nebraska: Candidates share the college votes by their popular vote rates, round the number to an integer. (In real life, using the 'congressional district method', these states allocate two electoral votes to the state popular vote winner, and then one electoral vote to the popular vote winner in each Congressional district (2 in Maine, 3 in Nebraska). Maine enacted this rule in advance of the 1972 presidential election, while Nebraska enacted it starting with the 1992 election.
'Winner takes it all' for all other states.
The result of electoral college votes sightly differ from the official results. However, it does not change the outcome to make the elected-president.

The data for the election this year is incompleted, some states are still carry on counting ballots, so this result is NOT the official outcome. (However, we have already known who won!)

In [ ]:
#Dropping unnecessary variables in elections dataset
elections.drop(['state_fips','state_cen', 'office', 'writein','version','notes','state_ic'],axis = 'columns',inplace = True)
elections.head()

In [ ]:
#Getting a dataframe which contains State and State_po
state_name = elections['state'].unique()
state_code = elections['state_po'].unique()
state_name = pd.DataFrame(state_name, columns=['state'])
state_code = pd.DataFrame(state_code, columns = ['state_po'])
state_namecode = pd.concat([state_name, state_code], axis = 1)

In [ ]:
#This year election is in another dataset, get the total votes to state-level and add the 'Year' column
this_year = this_year_data.groupby(['state', 'candidate','party'])['votes'].sum()
this_year = this_year.to_frame()
this_year = this_year.reset_index()
this_year['year'] = 2020

In [ ]:
#Add State_po to the dataframe of this year election
this_year_withcode = this_year.merge(state_namecode, on='state')
this_year_withcode.head()

In [ ]:
#Calculate total votes for each candidate in each state
totalvotes_df = this_year.groupby('state')['votes'].sum()
totalvotes_df = totalvotes_df.reset_index()
totalvotes_df.rename(columns = {'votes':'totalvotes'},inplace = True)

#Add total state-level votes to this year dataframe
this_year_withcode = this_year_withcode.merge(totalvotes_df, on = 'state')
this_year_withcode.rename(columns = {'votes':'candidatevotes'}, inplace = True)

#Rename some other columns to be similar with elections dataset
this_year_withcode['party'].replace({'REP':'republican', 'DEM':'democrat'},inplace = True)
this_year_withcode.head()

In [ ]:
#Reorder the arrange of this year dataframe
this_year_withcode = this_year_withcode[['year','state', 'state_po', 'candidate','party', 'candidatevotes','totalvotes']]

#Merge the result of previous elections with the election this year
elections_full = pd.concat([elections,this_year_withcode])
elections_full = elections_full.reset_index()

elections_full.drop(['index'], axis =1, inplace = True)
elections_full.head()

In [ ]:
#Generate a column with the unique code for state, each year, for later joining purpose
elections_full['for_join'] =elections_full['year'].astype(str) + elections_full['state_po']

#Get the results for Republican and Democrats separately, then later join them side by side
democrat = elections_full.query("party=='democrat'")
republican = elections_full.query("party=='republican'")
df = republican.merge(democrat, on = 'for_join')

df.head()

In [ ]:
#Dropping duplicated columns and rename columns for using
df.drop(['year_y','state_y','state_po_y','party_x','party_y','totalvotes_x'], axis = 1, inplace = True)
df.rename(columns = {'year_x':'Year', 'state_x':'State','state_po_x':'state_po', 'candidate_x':'Rep_Candidate',
                    'candidatevotes_x':'Rep_votes','candidate_y':'Dem_Candidate', 'candidatevotes_y':"Dem_votes",
                     'totalvotes_y':'Total_votes'}, inplace = True)

In [ ]:
#Adding electrol college votes to each states

electrol.rename(columns={'STATE':'state_po'}, inplace = True)
df = df.merge(electrol, on = 'state_po')
df.head()

In [ ]:
#Checking if all candidate names are inified and no duplicated rows

print(df.groupby('Rep_Candidate')['Rep_Candidate'].count())
print(df.groupby('Dem_Candidate')['Dem_Candidate'].count())

In [ ]:
#Now,in df, there is only one row represent for one election in one state, 
# so the 'fo_join' value should be unique in order to ensure there are more than 1 row of result for an election in a state
#Filter for duplicated values 
for_filter = df.groupby('for_join')['for_join'].count() !=1
for_filter = for_filter.to_frame()
for_filter[for_filter['for_join']==True]

In [ ]:
#Check the information of dupilcated values 
print(df[df['for_join']=='2004MD'])
print(df[df['for_join']=='2016AZ'])
print(df[df['for_join']=='2016MD'])

In [ ]:
#Unify name for Mitt Romney, John Kerry and drop duplicated values 

df['Rep_Candidate'].replace({'Mitt, Romney':'Romney, Mitt'},inplace = True)
df['Dem_Candidate'].replace({'Other':'Kerry, John'},inplace = True)
df.drop([35,249, 253, 254,255],inplace = True)

In [ ]:
#Creating new ratio columns for competing votes for each party

df['Rep_rate']=df['Rep_votes']/df['Total_votes']
df['Dem_rate']= df['Dem_votes']/df['Total_votes']
df['Rep_Win?']= df['Rep_rate'] - df['Dem_rate']



In [ ]:
#Separate the data to 2 parts due to the difference in electoral college votes policies

partial = pd.concat([df[df['state_po']=='ME'], df[df['state_po']=='NE']])
whole = df[df['state_po']!='ME']
whole = whole[whole['state_po']!='NE']

In [ ]:
#Calculate electrol college votes for candidates in ME AND NE
partial['Dem_electrolvote'] = round(df['Dem_rate']*df['ELECTORAL VOTE'],0)
partial['Rep_electrolvote'] = round(df['Rep_rate']*df['ELECTORAL VOTE'],0)

In [ ]:
#Calculate electrol college votes for candidates in "Winner takes it all" - states
whole['Dem_electrolvote'] = np.where(whole['Rep_Win?']>0, 0, whole['ELECTORAL VOTE'])
whole['Rep_electrolvote'] = np.where(whole['Rep_Win?']<0, 0, whole['ELECTORAL VOTE'])
 

In [ ]:
#Join the two type of electrol votes policy states
full_result = pd.concat([partial, whole])

#Get the table to make the heat map
wise_party = full_result[['Year', 'State', 'Rep_Win?']]

#Convert to pivot table
wise_party = wise_party.pivot(index = 'State', columns = 'Year', values = 'Rep_Win?')

In [ ]:
#Plotting the political trend in states 
g = sns.heatmap(wise_party, vmax = 0.3, vmin = -0.3, cmap = 'coolwarm', center = 0, yticklabels = True)
g.set_title('US State-level political trend over elections', size = 16, y = 1.05)
g.figure.set_size_inches(12, 15)
fig = g.get_figure()
fig.savefig('State-levelheatmap.jpg')

From here, we can see why they call 'Republican states' and 'Democrat states'. There are states always red or always blue since the 1976 election. However, the period of 1980 to 1992, we can see most states in America turned to the warm-tone. It was the recognition of the Republican credentials, under the leading of Ronal Reagan and Reagan Revolution.

The next 2 elections which brought the presidency to Bill Clinton, the popularity between 2 parties were quite competitive, which represents by the neutral colour tones.

Getting closer to the present, American politic opinions getting more divided. Many states get heavy red, many others get heavy blue, while some stay "swing", which are: Virginia, North Carolina, Pennsylvania, Ohio, Iowa, Georgia, Florida and Arizona. That's why they call these are 'battleground' for presidential campaigns because, with only one more popular votes, a candidate can get all college votes.

In [ ]:
#Calculate for the president-elected 
national_level = full_result.groupby(['Year','Rep_Candidate','Dem_Candidate'])['Rep_votes',
                                                                                    'Dem_votes','Dem_electrolvote',
                                                                                    'Rep_electrolvote','Total_votes'].sum()
national_level=national_level.reset_index()

#Creating new columns for comparing ratios

national_level['Rep_rate']=national_level['Rep_votes']/national_level['Total_votes']
national_level['Dem_rate']= national_level['Dem_votes']/national_level['Total_votes']
national_level['Rep_Win?']= (national_level['Rep_rate'] - national_level['Dem_rate'])*1000
national_level['Rep_Win?votecount'] = national_level['Rep_votes'] - national_level['Dem_votes']

In [ ]:
#President-elected is the one who gets over the threshold 270 electrol college votes

national_level['President-Elected']=np.where(national_level['Rep_electrolvote']>270, 
                                                  national_level['Rep_Candidate'], national_level['Dem_Candidate'])

In [ ]:
#The result may have minor vary from the official result due to the authenticity of the data and the policy changing at
#certain states/counties 
national_level['Presidentforshort'] = ['Carter', 'Reagan', 'Reagan ', 'Bush.', 'Clinton', 
                                            'Clinton ', 'Bush', 'Bush ', 'Obama', 'Obama ', 'Trump', 'Biden']
national_level.to_csv('National_level_president_election.csv')

In [ ]:
national_level

In [ ]:
sns.set_style('whitegrid')
g = sns.barplot(x = 'Presidentforshort', y = 'Rep_Win?', data = national_level, hue = 'Rep_Win?', palette='coolwarm', dodge = False)
g.legend([])
g.figure.set_size_inches(12, 8)
g.set_title('Politic Side and Popular vote rate margin', size = 16, y = 1.05)
g.set(ylabel = 'National-level Popular vote margin - per thousand', 
     ylim=(-200, 200))
g.set_xlabel('')
fig = g.get_figure()
fig.savefig('Politicside.jpg')

To the national-scale, after the presidency of (father) Bush, it seems to be more challenging for any Republican candidate to get more popular votes. Significantly, in 2000, Bush (son) won the White House without winning the public. He was behind Gore Al ONLY 46 thousands votes. History repeated in the 2016 election with 8 times larger margin, Trump won the presidency with almost 3 million votes behind Clinton.

Clearly, getting more public votes is not the key to the White House.

In [ ]:
#Plotting the differences 
my_range=range(1,len(national_level.index)+1)

In [ ]:
#Plotting the margin of popular votes
import seaborn as sns
from matplotlib.pyplot import figure
figure(num=None, figsize=(12, 6), dpi=80, facecolor='w', edgecolor='k')
sns.set_style('whitegrid')

plt.hlines(y = my_range, xmin = national_level['Rep_votes'], 
           xmax =national_level['Dem_votes'], color = 'green', alpha = 0.4)
plt.scatter(national_level['Rep_votes'], my_range, color = 'red', alpha = 0.8)
plt.scatter(national_level['Dem_votes'], my_range, color = 'blue', alpha = 0.8)
plt.legend(['Margin','Republican','Democrats'])
plt.yticks(np.arange(1, 13, step=1), 
           labels = (national_level['Year']))
plt.ylabel('Year', size = 14)
plt.xlim(20000000,80000000)
plt.xlabel('Votes x 10 millions', size = 14)
plt.title('Republican vs Demorats poloular votes over elections', size = 16, y = 1.05)
plt.savefig('Popularvote_margin.jpg')

In [ ]:
#Plotting the margins of electrol college votes
figure(num=None, figsize=(12, 6), dpi=80, facecolor='w', edgecolor='k')
plt.hlines(y = my_range, xmin = national_level['Rep_electrolvote'], 
           xmax =national_level['Dem_electrolvote'], color = 'green', alpha = 0.4)
plt.scatter(national_level['Rep_electrolvote'], my_range, color = 'red', alpha = 0.8)
plt.scatter(national_level['Dem_electrolvote'], my_range, color = 'blue', alpha = 0.8)
plt.legend(['Margin','Republican','Democrats'])
plt.yticks(np.arange(1, 13, step=1), 
           labels = (national_level['Year']))
plt.ylabel('Year', size = 14)
plt.xlabel('Electrol College votes', size = 14)
plt.title('Republican and Demorats Electrol college vote margin over elections', size = 16, y = 1.05)
plt.savefig('Electrol-collegevote-margin.jpg')

In [ ]:
#Get the turn-out ratios
turnout['VEP/Pop_rate'] = turnout['VEP']/turnout['Population']
turnout['Turnout/Pop_rate'] = turnout['Turnout']/turnout['Population']
turnout['Turnout/VEP_rate'] = turnout['Turnout']/turnout['VEP']

In [ ]:
#Plotting Voter turnout overtime
sns.set_style('white')
fig, ax = plt.subplots(figsize=(15, 7))
ax.axvspan(1929,1933,color='steelblue',alpha=0.1)
ax.axvspan(1937,1938,color='steelblue',alpha=0.1)
ax.axvspan(1945,1945.7,color='steelblue',alpha=0.1)
ax.axvspan(1948,1949,color='steelblue',alpha=0.1)
ax.axvspan(1953,1954,color='steelblue',alpha=0.1)
ax.axvspan(1957,1958,color='steelblue',alpha=0.1)
ax.axvspan(1960,1961,color='steelblue',alpha=0.1)
ax.axvspan(1969,1970,color='steelblue',alpha=0.1)
ax.axvspan(1973,1975,color='steelblue',alpha=0.1)
ax.axvspan(1980,1982,color='steelblue',alpha=0.1)
ax.axvspan(1990,1991,color='steelblue',alpha=0.1)
ax.axvspan(2000,2001,color='steelblue',alpha=0.1)
ax.axvspan(2007,2009,color='steelblue',alpha=0.1)
ax.axvspan(2019.5,2021.5,color='steelblue',alpha=0.1)
ax.plot(turnout['Year'], turnout['Turnout/Pop_rate'], color = 'green')
ax.plot(turnout['Year'], turnout['Turnout/VEP_rate'], color = 'orange')
ax.legend(['Turnout/Pop_rate','Turnout/VEP_rate'])
ax.set_title('Voter turnout rates over years', size = 16, y = 1.05)
ax.set_xticks(turnout['Year'])
plt.annotate('Recession', xy = (2021, 0.4), xytext =(2025, 0.5),
            arrowprops = {'facecolor':'black', 'width':3, 'shrink':0.03}, size = 12)
plt.savefig('Turnout-rate&Recession.jpg')


The voter turnout rate in America has fluctuated over time but in general, it has been increasing. It's heavily relied on the voting policies change in America. The fact is, the voting right has reached closer to each of every group of society.

Does the voter turn-out rate have a strong correlation to economy recession? This question perhaps needs a much deeper analysis to answer. Nonetheless, this chart does show some relation between the increasing number of people who vote and the economic situation, especially from 1990.

In [ ]:
dmg=dmg[dmg['Year']>1970]

In [ ]:
#Plotting USA economy and society over times
fig, bx = plt.subplots(figsize=(15, 7))
bx.axvspan(1973,1975,color='grey',alpha=0.2)
bx.axvspan(1980,1982,color='grey',alpha=0.2)
bx.axvspan(1990,1990.5,color='grey',alpha=0.2)
bx.axvspan(2000,2000.5,color='grey',alpha=0.2)
bx.axvspan(2007,2009,color='grey',alpha=0.2)
bx.axvspan(2020,2022,color='grey',alpha=0.2)
bx.axvspan(1970,1976,color='red',alpha=0.05)
bx.axvspan(1976,1980,color='blue',alpha=0.05)
bx.axvspan(1980,1988,color='red',alpha=0.05)
bx.axvspan(1988,1992,color='red',alpha=0.05)
bx.axvspan(1992,2000,color='blue',alpha=0.05)
bx.axvspan(2000,2008,color='red',alpha=0.05)
bx.axvspan(2008,2016,color='blue',alpha=0.05)
bx.axvspan(2016,2020,color='red',alpha=0.05)
bx.axvspan(2020,2024,color='blue',alpha=0.05)
bx.plot(dmg['Year'], dmg['Unemployment_rate'], color = 'green')
bx.plot(dmg['Year'], dmg['Annual_GDP_growth'], color = 'orange')
bx.legend(['Unemployment rate', 'Annual GDP growth'])
bx.set_title('US Politic background with GDP growth and Unemployment rates', size = 16, y = 1.05)
bx.set_ylabel('Rate %', size = 14)
plt.text(1973, -3, 'Ford', size = 12)
plt.text(1977, -3, 'Carter', size = 12)
plt.text(1983, -3, 'Reagan', size = 12)
plt.text(1989, -3, 'Bush', size = 12)
plt.text(1995, -3, 'Cliton', size = 12)
plt.text(2003, -3, 'Bush', size = 12)
plt.text(2011, -3, 'Obama', size = 12)
plt.text(2017, -3, 'Trump', size = 12)
plt.text(2021, -3, 'Biden', size = 12)
plt.annotate('Recession', xy = (2021, 4), xytext =(2025, 8),
            arrowprops = {'facecolor':'black', 'width':3, 'shrink':0.03}, size = 12)
plt.savefig('Politic_and_economy.jpg')

Recessions may not fully urge people to go voting, but it has been the game-changer in deciding if the ruling party will continue to stay in power.

Republican Reagan replaced Democrat Carter in the Whitehouse after the beginning of the big dip since 1980. Democrat Clinton replaced Republican Bush after the 1990 recession. The initial issue Obama had to face is the economic recovery from the great recession since 2007, in the late period of Bush. And this year, Democrat got back the White House from Trump during the century global health pandemic with the deepest deep of the GDP growth.

In conclusion, because there are only two big parties which can able to govern the country, American has not always voted to choose the president but the party. And more importantly, they vote to decide whether to continue the direction of the current ruling party or to change.